In [1]:
import pandas as pd 
import glob

In [2]:
#READING ALL EXCELS
path='/datc/opschaler/smartmeter_data'
filenames = glob.glob(path + "/*.csv")

In [20]:
dfs=[]
for f in filenames:
    df = pd.read_csv(f,delimiter=';')
    dfs.append(df)

In [21]:
dfs[0]

,Timestamp,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasTimestamp,gasMeter
0,2017-03-07 07:35:16,2402.278,0.0,1872.316,0.0,2576.0,0.0,2017-03-07 07:00:00,1017.653
1,2017-03-07 07:35:26,2402.285,0.0,1872.316,0.0,2580.0,0.0,2017-03-07 07:00:00,1017.653
2,2017-03-07 07:35:36,2402.292,0.0,1872.316,0.0,1167.0,0.0,2017-03-07 07:00:00,1017.653
3,2017-03-07 07:35:46,2402.293,0.0,1872.316,0.0,542.0,0.0,2017-03-07 07:00:00,1017.653
4,2017-03-07 07:35:56,2402.295,0.0,1872.316,0.0,638.0,0.0,2017-03-07 07:00:00,1017.653
5,2017-03-07 07:36:06,2402.297,0.0,1872.316,0.0,651.0,0.0,2017-03-07 07:00:00,1017.653
6,2017-03-07 07:36:16,2402.298,0.0,1872.316,0.0,612.0,0.0,2017-03-07 07:00:00,1017.653
7,2017-03-07 07:36:26,2402.300,0.0,1872.316,0.0,545.0,0.0,2017-03-07 07:00:00,1017.653
8,2017-03-07 07:36:56,2402.305,0.0,1872.316,0.0,678.0,0.0,2017-03-07 07:00:00,1017.653
9,2017-03-07 07:37:06,2402.307,0.0,1872.316,0.0,729.0,0.0,2017-03-07 07:00:00,1017.653
